In [1]:
import pandas as pd
import numpy as np
import json

models = []

lr = []
svm = []
dt = []
rf = []
gbt = []

path_compo = 'D:data/component_1'

for i in range(20):
    
    result_lr = pd.read_csv(path_compo + '/lr/' + str(i) + '.csv', index_col=0)
    result_svm = pd.read_csv(path_compo + '/svm/' + str(i) + '.csv', index_col=0)
    result_dt = pd.read_csv(path_compo + '/dt/' + str(i) + '.csv', index_col=0)
    result_rf = pd.read_csv(path_compo + '/rf/' + str(i) + '.csv', index_col=0)
    result_gbt = pd.read_csv(path_compo + '/gbt/' + str(i) + '.csv', index_col=0)
    
    lr.append(result_lr)
    svm.append(result_svm)
    dt.append(result_dt)
    rf.append(result_rf)
    gbt.append(result_gbt)
    
models = [lr, svm, dt, rf ,gbt]

In [2]:
def evaluate(cc_pdf):
    
    set_pred = set()
    set_true = set()

    for compo in cc_pdf['component'].value_counts().index:
        temp_index = cc_pdf[cc_pdf['component'] == compo].index
        temp_len = len(temp_index)
        for i in range(temp_len):
            for j in range(temp_len):
                if i != j:
                    id_i = cc_pdf.loc[temp_index[i], 'id']
                    id_j = cc_pdf.loc[temp_index[j], 'id']
                    set_pred.update(((id_i, id_j), (id_j, id_i)))
                    
    for author in cc_pdf['author_id'].value_counts().index:
        temp_index = cc_pdf[cc_pdf['author_id'] == author].index
        temp_len = len(temp_index)
        for i in range(temp_len):
            for j in range(temp_len):
                if i != j:
                    id_i = cc_pdf.loc[temp_index[i], 'id']
                    id_j = cc_pdf.loc[temp_index[j], 'id']
                    set_true.update(((id_i, id_j), (id_j, id_i)))
                    
    TP = len(set_pred & set_true) / 2
    FP = len(set_pred - set_true) / 2
    FN = len(set_true - set_pred) / 2
    
    if (TP + FP) != 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
        
    if (TP + FN) != 0:
        recall = TP / (TP + FN)
    else:
        recall = 0
        
    if (precision + recall) != 0:
        F1 = 2 * precision * recall / (precision + recall)
    else:
        F1 = 0
    
    print('TP:{}, FP:{}, FN:{}, precision:{}, recall:{}, F1:{}'.format(TP, FP, FN, precision, recall, F1))
    
    return TP, FP, FN, precision, recall, F1

In [3]:
def result(models, scores, macro_score, i):
    
    TPs = []
    FPs = []
    FNs = []
    precisions = []
    recalls = []
    F1s = []
    
    for k in range(len(models[i])):
        
        print(k)
        TP, FP, FN, precision, recall, F1 = evaluate(models[i][k])
        
        scores[i] = scores[i].append(pd.DataFrame({'TP':[TP], 'FP':[FP], 'FN':[FN], 'precision':[precision], 'recall':[recall], 'F1':[F1]}, index = [author_name[k]]))
        
        TPs.append(TP)
        FPs.append(FP)
        FNs.append(FN)
        precisions.append(precision)
        recalls.append(recall)
        F1s.append(F1)
    
    macro_P = np.mean(precisions)
    macro_R = np.mean(recalls)
    Macro_F1 = np.mean(F1s)
    
    macro_score = macro_score.append(pd.DataFrame({'macro-P':[macro_P], 'macro-R':[macro_R], 'Macro-F1':[Macro_F1]}, index = [model_name[i]]))
    
    print('\n')
    print('macro-P {}, macro-R {}, Macro-F1 {}'.format(macro_P, macro_R, Macro_F1))
    
    return macro_score

In [4]:
model_name = ['lr', 'svm', 'dt', 'rf', 'gbt']
author_name = ['mei_xu', 'mian_chen', 'yong_qing_huang', 'yin_shi', 'lili_ma','kwok_fai_so', 'wen_chang_chen', 
             'mei_ling_chen', 'yue_tian', 's_lin', 'yongqing_li', 'junling_wang', 'chun_hung_lin', 'guotong_du',
             'jianqiang_yi', 'chih_ming_lin', 'weiming_zhu', 'hongbin_liang','li_zhu_wu', 'xiaobing_luo']

scores = []
macro_score = pd.DataFrame(columns=('macro-P', 'macro-R', 'Macro-F1'))

for i in range(len(models)):
    score = pd.DataFrame(columns=('TP', 'FP', 'FN', 'precision', 'recall', 'F1'))
    scores.append(score)

for i in range(len(models)):
    print(model_name[i])
    macro_score = result(models, scores, macro_score, i)
    if i != len(models) - 1:
        print('-' * 100)    

lr
0
TP:296.0, FP:3449.0, FN:48.0, precision:0.07903871829105474, recall:0.8604651162790697, F1:0.14477867449254098
1
TP:2998.0, FP:694.0, FN:7751.0, precision:0.8120260021668473, recall:0.2789096660154433, F1:0.4152067031369019
2
TP:1994.0, FP:1646.0, FN:4082.0, precision:0.5478021978021979, recall:0.32817643186306783, F1:0.4104569781803212
3
TP:2271.0, FP:1268.0, FN:2351.0, precision:0.6417066968070076, recall:0.4913457377758546, F1:0.5565494424702855
4
TP:3071.0, FP:419.0, FN:7971.0, precision:0.8799426934097421, recall:0.2781199058141641, F1:0.422653454445362
5
TP:2237.0, FP:1253.0, FN:1720.0, precision:0.640974212034384, recall:0.5653272681324235, F1:0.600778837115617
6
TP:3077.0, FP:365.0, FN:7240.0, precision:0.8939570017431726, recall:0.2982456140350877, F1:0.4472708772439857
7
TP:1204.0, FP:2191.0, FN:723.0, precision:0.354639175257732, recall:0.6248053969901401, F1:0.45246148064637354
8
TP:882.0, FP:2467.0, FN:759.0, precision:0.26336219767094654, recall:0.5374771480804388, F

TP:917.0, FP:2497.0, FN:724.0, precision:0.2685998828353837, recall:0.558805606337599, F1:0.362809099901088
9
TP:114.0, FP:3255.0, FN:3.0, precision:0.03383793410507569, recall:0.9743589743589743, F1:0.06540447504302925
10
TP:1612.0, FP:1628.0, FN:1285.0, precision:0.49753086419753084, recall:0.556437694166379, F1:0.525338113084569
11
TP:768.0, FP:2206.5, FN:596.0, precision:0.25819465456379226, recall:0.5630498533724341, F1:0.3540394145441974
12
TP:1421.0, FP:1553.0, FN:1320.0, precision:0.47780766644250167, recall:0.5184239328712149, F1:0.4972878390201225
13
TP:2785.0, FP:122.0, FN:6806.0, precision:0.9580323357413141, recall:0.2903763945365447, F1:0.4456713074091855
14
TP:2406.0, FP:324.0, FN:2208.0, precision:0.8813186813186813, recall:0.5214564369310793, F1:0.6552287581699346
15
TP:802.0, FP:1830.0, FN:751.0, precision:0.3047112462006079, recall:0.5164198325820991, F1:0.38327359617682194
16
TP:1232.0, FP:1283.0, FN:1238.0, precision:0.48986083499005967, recall:0.4987854251012146, 

In [5]:
scores[0]

,TP,FP,FN,precision,recall,F1
mei_xu,296.0,3449.0,48.0,0.079039,0.860465,0.144779
mian_chen,2998.0,694.0,7751.0,0.812026,0.278910,0.415207
yong_qing_huang,1994.0,1646.0,4082.0,0.547802,0.328176,0.410457
yin_shi,2271.0,1268.0,2351.0,0.641707,0.491346,0.556549
lili_ma,3071.0,419.0,7971.0,0.879943,0.278120,0.422653
kwok_fai_so,2237.0,1253.0,1720.0,0.640974,0.565327,0.600779
wen_chang_chen,3077.0,365.0,7240.0,0.893957,0.298246,0.447271
mei_ling_chen,1204.0,2191.0,723.0,0.354639,0.624805,0.452461
yue_tian,882.0,2467.0,759.0,0.263362,0.537477,0.353507
s_lin,114.0,3190.0,3.0,0.034504,0.974359,0.066647


In [6]:
macro_score

,macro-P,macro-R,Macro-F1
lr,0.596121,0.537223,0.478702
svm,0.588752,0.540731,0.477702
dt,0.608589,0.535525,0.482323
rf,0.588752,0.540731,0.477702
gbt,0.608589,0.535525,0.482323


In [7]:
path = 'D:data/score_1/'

for i in range(len(models)):
    scores[i].to_csv(path + model_name[i] + '.csv')
    macro_score.to_csv(path + 'macro_score.csv')